In [7]:
using Random, LaTeXStrings , ColorSchemes , Statistics, Plots, Distributions, LinearAlgebra, CSV, DataFrames

Initializing

In [5]:
mutable struct MDBox
    n ::Int
    L :: Float64
    T :: Float64
    xlist :: Vector{Float64}
    ylist :: Vector{Float64}
    axlist :: Vector{Float64}
    aylist :: Vector{Float64}
    V :: Float64
    K :: Float64
    E :: Float64
    P :: Float64
    TRange :: Int
    step :: Float64
    dmax :: Float64
end

function Init(n, L ,T)
    xlist =  zeros(n)
    ylist = zeros(n)                                # Initializing rs
    for i in 1:(2*Int(√(n)))                          # Align Left     
        for j in 1:(Int(√(n)÷2))
        xlist[(j)+(i-1)*Int(√(n)/2)] = j*L/(2*(sqrt(n)/2+1))
        ylist[(j)+(i-1)*Int(√(n)/2)] = i*L/(2*sqrt(n)+1)
        end
    end
    axlist = zeros(n)
    aylist = zeros(n)
    V = 0.0 
    K = 0.0
    E = 0.0
    P=0.0
    TRange = 100000
    step = 0.01
    dmax = 1
   return MDBox(n ,L , T  , xlist , ylist , axlist, aylist,V , K , E  , P,TRange, step, dmax )
end


Init (generic function with 1 method)

Potential Calculator

In [3]:

function Potential(box)                 #This function is completely similar to my Forces() duntion in MD code, but it only calculates potentials. periodic condition is also applied.

    ϵ = 0
    box.V = 0.0

    for i in 1:box.n 
        xrepeati = abs(box.L-abs(box.xlist[i]))
        yrepeati = abs(box.L-abs(box.ylist[i]))
        if ((box.xlist[i]<0) || (box.xlist[i]>box.L))           #Periodic boundary condition 
            box.xlist[i] = xrepeati
        end
        if ((box.ylist[i]<0) || (box.ylist[i]>box.L))
            box.ylist[i] = yrepeati
        end
        for j in 1:i
            rreal = 0.0
            if (i==j)
                continue    
            end
            if box.xlist[i]<(box.L/4) && box.ylist[i] < (box.L/4)               # Boundary conditions for finding the nrighbours in 2.5σ. We only check the two nearest side of the box for each particle based of it's position in the box.
                xrepeatj = box.xlist[j] -  box.L
                yrepeatj = box.ylist[j] -  box.L
            elseif box.xlist[i]<(box.L/4) && box.ylist[i] > (box.L/4)
                xrepeatj = box.xlist[j] -  box.L
                yrepeatj = box.ylist[j] +  box.L
            elseif box.xlist[i]>(box.L/4) && box.ylist[i] < (box.L/4)
                xrepeatj = box.xlist[j] +  box.L
                yrepeatj = box.ylist[j] -  box.L
            else
                xrepeatj = box.xlist[j] +  box.L
                yrepeatj = box.ylist[j] +  box.L
            end  

            deltax = box.xlist[j]-box.xlist[i]                  # Calculating different distances for each two neighbours (including the mirrors)
            deltay = box.ylist[j]-box.ylist[i]
            deltaxrepeat = xrepeatj - box.xlist[i] 
            deltayrepeat = yrepeatj - box.ylist[i] 
            r = sqrt((deltax)^2 +(deltay)^2)
            rhorizontal = sqrt((deltaxrepeat)^2 +(deltay)^2)
            rvertical = sqrt((deltax)^2 +(deltayrepeat)^2)
            rdiagonal = sqrt((deltaxrepeat)^2 +(deltayrepeat)^2)

            if r<2.5                                            # If the j particle is not a neighbour for the i particle, the mirror might be. so the distance is set to be the relating one.
                                                                # A relatively small ϵ was first used for not letting the force to diverge. at last it was set to zero.
                rreal = r+ϵ
            elseif rhorizontal<2.5
                rreal = rhorizontal+ϵ
                deltax = deltaxrepeat
            elseif rvertical<2.5
                rreal = rvertical+ϵ
                deltay = deltayrepeat
            elseif rdiagonal<2.5
                rreal = rdiagonal+ϵ
                deltax = deltaxrepeat
                deltay = deltayrepeat
            else
    
                    continue
            end
            r2 = rreal*rreal
            r6 = r2*r2*r2
            r12 = r6*r6
            v = 4*((1/r12)-(1/r6))                    # Calculating Lennard Jones potential
            box.V += v
            
            F = -(4/(r2))*((12/(r12))-(6/(r6)))
            ax = F *deltax                                   # The force decomposed to x and y direction
            ay = F *deltay
            box.axlist[i] += ax
            box.axlist[j] += -ax
            box.aylist[i] += ay
            box.aylist[j] += -ay

        end
    end

end


Potential (generic function with 1 method)

Mont Carlo step

In [4]:
function montcarlo(box)                                                         # defining a function to perform a Monte Carlo step
    count = 0                                                                   # initialize a counter for the number of accepted moves
    randindex = rand(1:box.n)                                                   # choosing a random particle to move
    Rrand = box.dmax * rand()                                                           # We want the random displacement vector to be symmetrical in polar coordinates, so it should be calculated using r and theta random coefficents.
    θrand = 2*pi*rand()
    newX = box.xlist[randindex] +   Rrand*sin(θrand)                            # proposing a new x coordinate by displacing it using rrand and thetarand
    newY = box.ylist[randindex] +   Rrand*sin(θrand)                            # proposing a new y coordinate by displacing it using rrand and thetarand
    tempbox.xlist[randindex] = newX                                             # updating the x coordinate of the particle in the temporary box
    tempbox.ylist[randindex] = newY                                             # updating the y coordinate of the particle in the temporary box
    oldX =  box.xlist[randindex]                                                # saving the old x coordinate of the particle
    oldY =  box.ylist[randindex]                                                # saving the old y coordinate of the particle
    newPotential = Potential(tempbox)                                           # calculating the new potential energy of the system in the temporary box
    ΔV = tempbox.V - box.V                                                      # calculating the energy difference due to the move
    probability = exp(-ΔV/(box.T))                                              # calculating the acceptance probability 
        if probability >1 || rand()< probability                                # accepting the move if it lowers the energy or with a probability of exp(-ΔV/T) if it raises the energy
            box.xlist = tempbox.xlist                                           # updating the x coordinates of the particles in the original box
            box.ylist = tempbox.ylist                                           # updating the y coordinates of the particles in the original box
            count = count + 1                                                   # incrementing the counter for the number of accepted moves
        else                                                                    # rejecting the move and restore the old position
            tempbox.xlist[randindex] = oldX                                     # restoring the x coordinate of the particle in the temporary box
            tempbox.ylist[randindex] = oldY                                     # restoring the y coordinate of the particle in the temporary box
        end

    MCMDInit.P = MCMDInit.n * MCMDInit.T / (MCMDInit.L*MCMDInit.L) + (dot(MCMDInit.xlist,MCMDInit.axlist) + dot(MCMDInit.ylist ,MCMDInit.aylist))/(3*(MCMDInit.L*MCMDInit.L))      #using virial
    MCMDInit.E = MCMDInit.T+ MCMDInit.V

end


montcarlo (generic function with 1 method)

In [8]:
MCMDInit = Init(100,30,1)
tempbox = deepcopy(MCMDInit)                                                         # making a copy of the box to store the new position and potential energy
Xs = zeros(MCMDInit.n, MCMDInit.TRange)
Ys = zeros(MCMDInit.n, MCMDInit.TRange)
Ks = zeros(MCMDInit.TRange)
Es = zeros(MCMDInit.TRange)
Vs = zeros(MCMDInit.TRange)
Ps = zeros(MCMDInit.TRange)

 for i in 1:MCMDInit.TRange
    montcarlo(MCMDInit)
    Xs[:,i] = MCMDInit.xlist
    Ys[:,i] = MCMDInit.ylist
    Ks[i]  = MCMDInit.K
    Es[i] = MCMDInit.E
    Vs[i] = MCMDInit.V
    Ps[i] = MCMDInit.P
 end


Animation

In [9]:

MdAnime = @animate for L in 1:MCMDInit.TRange
    scatter(Xs[:,L],Ys[:,L], xlim = [0,30], ylim = [0,30])
end


Animation("C:\\Users\\Nahal\\AppData\\Local\\Temp\\jl_JgV9Di", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "099991.png", "099992.png", "099993.png", "099994.png", "099995.png", "099996.png", "099997.png", "099998.png", "099999.png", "100000.png"])

In [16]:
gif(MdAnime,"McMdAnime", fps = 30)
;

N(t)

In [15]:
count = zeros(MCMDInit.TRange)
for L in 1:10
for i in 1:MCMDInit.n
    if Xs[i,L] < (MCMDInit.L/2)
        count[L] +=1
    end
end
end
plot(count, label = false)
;

Energy

In [13]:
plot(Es, label = "Es", lable = "E")
plot!(Ks, label = "Ks", lable = "K")
plot!(Vs , label = "Vs", lable = "U")
plot!(xaxis = "t", yaxis = "E")
;

Pressure

In [14]:
plot!(Ps, label = "Ps", lable = "K")
;

Van der Waals

In [ ]:
simrun = 500000
Paverage =zeros(100)
Taverage = zeros(100)
Tlist = zeros(simrun)
Plist = zeros(simrun)

for i in 1:100
    mddata =  Init(100,30,1+(i/50))
    for t in 1:simrun
        montcarlo(mddata)
        Tlist[t] = mddata.T
        Plist[t] = mddata.P
    end
    Taverage[i] = movingaverage(Tlist,Int(0.7*simrun ))[end]
    Paverage[i] = movingaverage(Plist,Int(0.7*simrun ))[end]
end


In [ ]:
# plot(Taverage,Paverage, label = false)
# plot!(xaxis = "T" , yaxis = "P")
plot(x->0.02 + 0.11*x)

In [ ]:
A= [ones(100) Taverage]
para = A \ Paverage

Phase Transition

In [ ]:
mdscale = Init(100,30,5)
Ts = zeros( simrun)
Ps = zeros(simrun)

for t in 1:simrun
    montcarlo(mdscale)
    if t%100 == 1
        mdscale.vxlist = 0.4 * mdscale.vxlist
        mdscale.vylist = 0.4 * mdscale.vylist

    end
    Ts[t] = mdscale.T
    Ps[t] = mdscale.P
end

In [ ]:
Tsmooth = movingaverage(Ts,5000)
Psmooth = movingaverage(Ps,5000)
plot(Tsmooth,Psmooth, legend= false)
plot!(xaxis = "T", yaxis = "P")